# Project Euler 182 - RSA encryption

- Author: Philippe Marchandise

Project Euler is a series of challenging mathematical/computer science problems that will require more than just mathematical insights to solve. Although mathematics will help arriving at elegant and efficient methods, programming will be required to solve most problems. Once well optimized, those problems can be solved in less than a minute by any computer. Without optimization, they usually cannot be solved in a reasonable time.

The difficulty ranges from 5% to 100%. The following is a __60% difficulty__ problem.

To this date (28/01/20), I have __completed 125 problems__ on Project Euler. They required solid mathematical reasoning and advanced optimization techniques.


## The problem

The RSA encryption is based on the following procedure:

Take two distinct primes p and q and let $\ n=pq\ $ and $\ \phi = (p-1)(q-1)\ $.
Find an integer $ e, 1<e<\phi \ $, such that $ \textrm{gcd}(\ e,\phi\ )=1\$.

A message in this system is a number in the interval $[0,n-1]$. To encrypt the message $m,\ c=m^e \ (\textrm{mod}\ n)$ is calculated.

To decrypt the text, calculate d such that $ed=1\ (\textrm{mod}\ \phi)$, then for each encrypted message c, calculate $m=cd\ (\textrm{mod}\ n)$.

There exist values of $e$ and $m$ such that $m^e\ (\textrm{mod}\ n )=m$.
We call them unconcealed messages. An issue when choosing $e$ is that there should not be too many unconcealed messages.

For instance, let $p=19$ and $q=37$. Then $n=703$ and $\phi = 648$
If we choose $e=181$, then, although $\textrm{gcd}(181,648)=1$, it turns out that all possible messages
$m\ (0 \leq m \leq n-1)$ are unconcealed.

For any valid choice of e there exist some unconcealed messages.
It's important that the __number of unconcealed messages is at a minimum__.

$\\ $

Choose $p=1009$ and $q=3643$. Find the sum of all values of $e$ so that the number of unconcealed messages is at a minimum.

## I - Complexity estimation

$ 1 < e < \phi(p,q)\, \ \ \ \ \ \ \ \ \ \ \ \ \phi(p,q) = (p-1)(q-1) = 1008 \times 3642 = 3671136$

Hence, there are $3\ 671\ 134$ possible values for $e$. Let's see how many satisfy $\ \textrm{gcd}(\ e, \phi\ ) = 1 )$

($\phi$ being even, we can ignore even values of $e$)

In [1]:
from math import gcd

p = 1009; q = 3643; phi = (p-1)*(q-1)

total = 0
for e in range(3, phi, 2):
    if gcd(e, phi) == 1: total += 1
total

1047167

There are $1\ 047\ 167$ possible values for $e$

For each one, we need all the values of $\ m^e\ \textrm{mod}\ n\ $ for $ 0 \leq m \leq n-1 $, so $3\ 675\ 787$ values to exponentiate.

Hence the complexity of this brute force approach is about $1\ 047\ 167 \times 3\ 675\ 787 = 3\ 849\ 162\ 845\ 429$ operations.

With $10^7$ operations per second, this would take 107 hours at best. Hence, it is not doable.

## II - Looking at solutions: is there an easy pattern?

Let's compute the solutions for the first few values of $e$

In [2]:
def getValues(elimite = 25):
    p = 1009; q = 3643; phi = (p-1)*(q-1)
    n = p*q

    values = []
    for e in range(2, elimite):

        if gcd(e, phi) == 1:

            values += [[e]]
            for m in range(0, n):

                # more efficient exponentiation when the modulo is applied at each step
                c = 1
                for i in range(e):
                    c = c*m % n

                if c == m: values[-1] += [m]
    return values

values = getValues()
print( "e:{} m:({})".format(values[0][0], ', '.join(map(str,values[0][1:])) )  )

e:5 m:(0, 1, 116575, 346086, 462662, 1548275, 1664850, 1664851, 2010936, 2010937, 2127512, 3213125, 3329701, 3559212, 3675786)


A lot of the solutions are __almost multiple of p or q__ (+/- 1). Are all the solutions like this?

In [3]:
v = values[0]
for i in v[1:]:
    if   i % p == 0   or i % q == 0  : pass
    elif i % p == 1   or i % q == 1  : pass
    elif i % p == p-1 or i % q == q-1: pass
    else: print('No', i)

True for e = 5, what about the others?

In [4]:
for v in values:
    print('e =',v[0], '\tcard = ', len(v)-1)
    for i in v[1:]:
        if   i % p == 0   or i % q == 0  : pass
        elif i % p == 1   or i % q == 1  : pass
        elif i % p == p-1 or i % q == q-1: pass
        else:
            print(v[0], 'No', i)
            break

e = 5 	card =  15
e = 11 	card =  9
e = 13 	card =  91
13 No 28721
e = 17 	card =  51
e = 19 	card =  133
19 No 33210
e = 23 	card =  9


Worked well until e = 13.

For $\ e = 13,\ 40/91 \ $ solutions don't verify the hypothesis. For $e = 19\ $, that's $64/133$. It seems to be about half but nothing obvious.

More analysis confirmed that those exceptions don't have any specificity: No common factor; no obvious link with n, $\phi$ or e; no specific period... They are pretty much unpredictable.

Is there an obvious __relation between the number of solution and $e$__?

In [5]:
for v in values: print("e = {:2}, v = {:3}".format(v[0], len(v)))

e =  5, v =  16
e = 11, v =  10
e = 13, v =  92
e = 17, v =  52
e = 19, v = 134
e = 23, v =  10


They are all even, other than that nothing obvious..

Maybe __multiple of e +/- 1 ?__

In [6]:
for v in values:
    l = len(v)
    if   (l-1) % v[0] == 0: print(v[0], -1)
    elif (l+1) % v[0] == 0: print(v[0],  1)
    elif  l    % v[0] == 0: print(v[0],  0)
    else: print(v[0], 'no')

5 -1
11 1
13 -1
17 -1
19 -1
23 no


No obvious pattern there neither... Let's not waste too much time here and tackle the theory

## III - Theory

When a message is not affected by the RSA encryption, it is called a fixed point. Such a message $m$ has the property $m^e \equiv m\ (\textrm{mod}\ n)$


To find all of those messages, we can use the Chinese Remainder Theorem (CRT). Knowing $n = p \times q $ and $p, q$ are coprime. we now have the system:

$$m^e \equiv m\ (\textrm{mod}\ p)$$

$$m^e \equiv m\ (\textrm{mod}\ q)$$

$\ $

Working on the first equation, we want to find the solutions $m$ such that:

$$m^e \equiv m\ (\textrm{mod}\ p)$$

$$\iff m\ (m^{e-1} - 1) \equiv 0\ (\textrm{mod}\ p)$$

$\ $

- If m is a factor of p: $\ \ \ \ m\ \equiv 0\ (\textrm{mod}\ p)$


- Else (if m and p are coprime), we can find a primitive root g such that $\ m = g^t\ $. Hence we want to find g such that:

$$g^{\ t\ e} \equiv g^{\ t}\ (\textrm{mod}\ p)$$

$$\iff t\ (e - 1) \equiv 0\ (\textrm{mod}\ \phi(p))$$


$\ \ \ \ \ \ $ Since p is prime, $\ \phi(p) = p-1$. Hence, this last equation has $\ \textrm{gcd}(\ e-1, p-1 )\$ solutions

$\ $

Hence, there are $\ \ 1 + \textrm{gcd}(\ e-1, p-1 )\ \ $ solutions to $\ \ m^e \equiv m\ (\textrm{mod}\ p)$.

By symmetry, there are $\ \ 1 + \textrm{gcd}(\ e-1, q-1 )\ \ $ solutions to $\ \ m^e \equiv m\ (\textrm{mod}\ q)$.

$\ $

Hence, the system has $\ \ (1 + \textrm{gcd}(\ e-1, p-1 )\ )\ (1 + \textrm{gcd}(\ e-1, q-1 )\ ) \ $ solutions.

## IV - Verification and last optimization

In [7]:
def getNsol(e, p=1009, q=3643):
    return (1 + gcd(e-1, p-1))*(1 + gcd(e-1, q-1))

for v in values: print(len(v[1:]), getNsol(v[0]), sep = '\t' )

15	15
9	9
91	91
51	51
133	133
9	9


Seems to work! Let's try to solve it now!

In [8]:
p=1009; q=3643; n = p*q; phi = (p-1)*(q-1)

mini = 9; sol = [];
for e in range(3, phi, 2):
    if gcd(e, phi) == 1:
        n = getNsol(e, p, q) 
        if n == mini:
            sol += [e]
        elif n < mini:
            mini = n
            sol = [e]
print(mini, sum(sol))

9 399788195976


The minimum seems to be 9, the answer is `399788195976`, obtained in `1.62 sec`: __this is the right answer!__

For fun, can we do even __faster__?

- If we directly sum the solutions, we avoid stocking 217,800 values. This takes us to `1.31s`

- The gcd account for `0.6s` of the `1.31s` taken. How to build a better loop? The values of $e$ that will yield `9` solutions follow the properties:

$$\textrm{gcd}(e, (p-1)(q-1)\ ) = 1, \ \ \ \ \textrm{gcd}(e-1, p-1\ ) = 2, \ \ \ \ \textrm{gcd}(e-1, q-1\ ) = 2 $$

We know that the prime factorization of $\phi$ is $2^5 . 3^3 . 7 . 607$

This means that $e$ cannot be a multiple of `2, 3, 7, 607`, and that $e-1$ is a multiple of `2` but not `3, 4, 7, 607`

So we get that $e$ must be odd, specifically  $ e \equiv 3\ (\textrm{mod}\ 4)$, and  $ e \equiv 2\ (\textrm{mod}\ 3)$

A generating sequence is hence `12 k + 11`: we just made the loop `6` times faster!

In [9]:
p=1009; q=3643; n = p*q; phi = (p-1)*(q-1)

s = 0
e = 11
while e < phi:
    if gcd(e, phi) == 1:
        if getNsol(e, p, q) == 9: s += e
    e += 12
print(mini, s)

9 399788195976


Wow! That's only `316ms` ! But can we do better? How about skipping the $\textrm{gcd}$ and `getNsol`?

If $\ e \equiv 11 (\textrm{mod}\ 12)\ $ and $\ e\ \textrm{mod}\ 7 \geq 2\ $ and $\ e\ \textrm{mod}\ 607 \geq 2\ $, then we are sure than $\textrm{gcd}(e, \phi) = 1\ $ and that there will be only 9 solutions.

In [11]:
p=1009; q=3643; n = p*q; phi = (p-1)*(q-1)

s = 0
e = 11
while e < phi:
    if e % 7 > 1:
        if e % 607 > 1:
            s += e
    e += 12
print(mini, s)

9 399788195976


`83ms`, that's what I call optimization!